<a href="https://colab.research.google.com/github/25b3nk/hf-nlp-course/blob/main/course/en/chapter5/section5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating your own dataset

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "csbhaskar95@gmail.com"
!git config --global user.name "25b3nk"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install requests

In [5]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [6]:
response.status_code

200

In [7]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7381',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7381/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7381/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7381/events',
  'html_url': 'https://github.com/huggingface/datasets/issues/7381',
  'id': 2815649092,
  'node_id': 'I_kwDODunzps6n02VE',
  'number': 7381,
  'title': 'Iterating over values of a column in the IterableDataset',
  'user': {'login': 'TopCoder2K',
   'id': 47208659,
   'node_id': 'MDQ6VXNlcjQ3MjA4NjU5',
   'avatar_url': 'https://avatars.githubusercontent.com/u/47208659?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/TopCoder2K',
   'html_url': 'https://github.com/TopCoder2K',
   'followers_url': 'https://api.github.com/users/TopCoder2K/followers',
   'f

In [8]:
from google.colab import userdata

GITHUB_TOKEN = userdata.get('GITHUB_API_KEY')  # Copy your GitHub token here
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [9]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Reached GitHub rate limit. Sleeping for one hour ...
Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [ ]:
!head -n 2 /content/datasets-issues.jsonl

{"url":"https:\/\/api.github.com\/repos\/huggingface\/datasets\/issues\/7378","repository_url":"https:\/\/api.github.com\/repos\/huggingface\/datasets","labels_url":"https:\/\/api.github.com\/repos\/huggingface\/datasets\/issues\/7378\/labels{\/name}","comments_url":"https:\/\/api.github.com\/repos\/huggingface\/datasets\/issues\/7378\/comments","events_url":"https:\/\/api.github.com\/repos\/huggingface\/datasets\/issues\/7378\/events","html_url":"https:\/\/github.com\/huggingface\/datasets\/issues\/7378","id":2802957388,"node_id":"I_kwDODunzps6nEbxM","number":7378,"title":"Allow pushing config version to hub","user":{"login":"momeara","id":129072,"node_id":"MDQ6VXNlcjEyOTA3Mg==","avatar_url":"https:\/\/avatars.githubusercontent.com\/u\/129072?v=4","gravatar_id":"","url":"https:\/\/api.github.com\/users\/momeara","html_url":"https:\/\/github.com\/momeara","followers_url":"https:\/\/api.github.com\/users\/momeara\/followers","following_url":"https:\/\/api.github.com\/users\/momeara\/fol

In [ ]:
df = pd.read_json("datasets-issues.jsonl", lines=True)

In [ ]:
df.columns

Index(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url',
       'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels',
       'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments',
       'created_at', 'updated_at', 'closed_at', 'author_association',
       'sub_issues_summary', 'active_lock_reason', 'body', 'closed_by',
       'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason',
       'draft', 'pull_request'],
      dtype='object')

In [ ]:
# print(len(df[df['closed_at'].isnull()]))
df['closed_at'] = df['closed_at'].fillna(pd.Timestamp('1970-01-01'))
# print(len(df[df['closed_at'].isnull()]))
df.to_json("datasets-issues-cleaned.jsonl", orient="records", lines=True)

In [ ]:
print(df.dtypes)

url                                      object
repository_url                           object
labels_url                               object
comments_url                             object
events_url                               object
html_url                                 object
id                                        int64
node_id                                  object
number                                    int64
title                                    object
user                                     object
labels                                   object
state                                    object
locked                                     bool
assignee                                 object
assignees                                object
milestone                                object
comments                                  int64
created_at                  datetime64[ns, UTC]
updated_at                  datetime64[ns, UTC]
closed_at                               

In [ ]:
import numpy as np
df['time_diff'] = df[df['closed_at'].notnull()]['closed_at'].astype(np.datetime64[ns]) - df[df['closed_at'].notnull()]['created_at']

In [ ]:
from datasets import load_dataset, Features, Value, Dataset

# features = Features({
#     'closed_at': Value('timestamp[s]', id=None),
#     'merged_at': Value('timestamp[s]', id=None)
# })

# issues_dataset = load_dataset("json", data_files="datasets-issues-cleaned.jsonl", split="train")
issues_dataset = Dataset.from_pandas(df)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 7314
})

In [ ]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/3765
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/3765.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/3765', 'merged_at': None, 'patch_url': 'https://github.com/huggingface/datasets/pull/3765.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/3765'}

>> URL: https://github.com/huggingface/datasets/issues/545
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/pull/526
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/526.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/526', 'merged_at': '2020-08-24T12:50:42Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/526.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/526'}



In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/7314 [00:00<?, ? examples/s]

In [ ]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

In [ ]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        # print(response.status_code)
        # print(response.json())
        return []
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

[]

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/7314 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
issues_with_comments_dataset.push_to_hub("github-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/25b3nk/github-issues/commit/8e3e1d5ac4d7d54a02f0cecab6f3c46f848a948d', commit_message='Upload dataset', commit_description='', oid='8e3e1d5ac4d7d54a02f0cecab6f3c46f848a948d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/25b3nk/github-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='25b3nk/github-issues'), pr_revision=None, pr_num=None)

In [ ]:
remote_dataset = load_dataset("25b3nk/github-issues", split="train")
remote_dataset

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.96M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7314 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request', 'is_pull_request'],
    num_rows: 7314
})

# Assignment
- Create a dataset with issues from ollama
- Upload the dataset
- Collate the dataset and load it for finetuning
- Train a multi-label classifier and do inference on new issues created or the test set